# Drone Imagery Search System

In [1]:
# Install required packages (only needed once)
!pip install flask pandas

## 1. Database Setup

In [2]:
import os
import re
from datetime import datetime
import sqlite3
import pandas as pd
from flask import Flask, request, jsonify, render_template_string
from IPython.display import HTML, display
import threading


def setup_database():
    """Create SQLite database and tables if they don't exist"""
    conn = sqlite3.connect("drone_folders.db")
    cursor = conn.cursor()

    # Create tables
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS professional (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        date TEXT,
        folder_path TEXT
    );
    """)

    cursor.execute("""
    CREATE TABLE IF NOT EXISTS personal (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        date TEXT,
        folder_path TEXT
    );
    """)

    conn.commit()
    conn.close()

    print("Database setup complete")

setup_database()

Database setup complete


## 2. Data Import from Google Drive

In [3]:
from google.colab import drive

def mount_google_drive():
    """Mount Google Drive to access files"""
    drive.mount('/content/drive')
    print("Google Drive mounted")

mount_google_drive()


def import_folder_data(folder_path):
    """Import folder names from specified path into database"""
    conn = sqlite3.connect("drone_folders.db")
    cursor = conn.cursor()

    inserted_pro = 0
    inserted_per = 0

    for name in os.listdir(folder_path):
        full_path = os.path.join(folder_path, name)
        if not os.path.isdir(full_path):
            continue

        # PROFESSIONAL FILES
        if name.startswith("DJI_") and "_" in name:
            try:
                parts = name.split("_")
                raw_date = parts[1]  # e.g. '202410171502'
                parsed_date = datetime.strptime(raw_date, "%Y%m%d%H%M").strftime('%Y-%m-%d')

                cursor.execute("SELECT 1 FROM professional WHERE name = ?", (name,))
                if cursor.fetchone() is None:
                    cursor.execute("INSERT INTO professional (name, date, folder_path) VALUES (?, ?, ?)",
                                 (name, parsed_date, full_path))
                    inserted_pro += 1
                    print(f"✅ Inserted professional folder: {name}")
            except Exception as e:
                print(f"❌ Failed to parse professional folder {name}: {e}")

        # PERSONAL FILES
        elif name.count('.') == 1:
            try:
                date_part = name.split('.')[0]
                day_and_rest = name.split('.')[1]
                day = day_and_rest[:2]
                rest = day_and_rest[2:]

                match = re.search(r'(20\d{2})', rest)
                year = match.group(1) if match else "2024"
                parsed_date = datetime.strptime(f"{year}-{date_part}-{day}", "%Y-%m-%d").strftime('%Y-%m-%d')

                cursor.execute("SELECT 1 FROM personal WHERE name = ?", (name,))
                if cursor.fetchone() is None:
                    cursor.execute("INSERT INTO personal (name, date, folder_path) VALUES (?, ?, ?)",
                                 (name, parsed_date, full_path))
                    inserted_per += 1
                    print(f"✅ Inserted personal folder: {name}")
            except Exception as e:
                print(f"❌ Failed to parse personal folder {name}: {e}")
        else:
            print(f"⚠️ Unrecognized format: {name}")

    conn.commit()
    conn.close()
    print(f"\n✅ Total professional folders inserted: {inserted_pro}")
    print(f"✅ Total personal folders inserted: {inserted_per}")

# Usage: (uncomment and modify path as needed)
folder_path = '/content/drive/Shareddrives/Drones - CSC 230/10-October'
import_folder_data(folder_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted

✅ Total professional folders inserted: 0
✅ Total personal folders inserted: 0


## 3. Run the Application


In [6]:
from flask import Flask, request, jsonify, render_template_string
from google.colab import output
from IPython.display import HTML, display
import threading
import sqlite3
from datetime import datetime
import time
import socket

app = Flask(__name__)

# Database connection
def get_db_connection():
    conn = sqlite3.connect("drone_folders.db")
    conn.row_factory = sqlite3.Row
    return conn

# HTML template (same as your original)
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Drone Imagery Search</title>
    <style>
        body { font-family: Arial, sans-serif; padding: 20px; }
        .search-form { background: #f5f5f5; padding: 20px; border-radius: 8px; margin-bottom: 20px; }
        .form-group { margin-bottom: 15px; }
        label { display: inline-block; width: 100px; font-weight: bold; }
        input, select { padding: 8px; border: 1px solid #ddd; border-radius: 4px; }
        button { background: #4CAF50; color: white; padding: 10px 15px; border: none; border-radius: 4px; cursor: pointer; }
        button:hover { background: #45a049; }
        .results { margin-top: 20px; }
        .result-item { border: 1px solid #ddd; padding: 15px; margin-bottom: 10px; border-radius: 4px; }
        .result-type { font-weight: bold; color: #fff; padding: 3px 8px; border-radius: 4px; }
        .professional { background: #2196F3; }
        .personal { background: #4CAF50; }
    </style>
</head>
<body>
    <h1>Drone Imagery Search</h1>
    <div class="search-form">
        <form id="searchForm">
            <div class="form-group">
                <label for="date_from">From Date:</label>
                <input type="date" id="date_from" name="date_from" required>
            </div>
            <div class="form-group">
                <label for="date_to">To Date:</label>
                <input type="date" id="date_to" name="date_to" required>
            </div>
            <div class="form-group">
                <label for="imagery_type">Type:</label>
                <select id="imagery_type" name="imagery_type">
                    <option value="all">All</option>
                    <option value="professional">Professional</option>
                    <option value="personal">Personal</option>
                </select>
            </div>
            <button type="submit">Search</button>
        </form>
    </div>
    <div id="results"></div>
    <script>
        document.getElementById('searchForm').addEventListener('submit', async (e) => {
            e.preventDefault();
            const formData = new FormData(e.target);
            const params = new URLSearchParams(formData);

            try {
                const response = await fetch('/search?' + params.toString());
                const data = await response.json();

                const resultsDiv = document.getElementById('results');
                if (data.error) {
                    resultsDiv.innerHTML = `<div style="color:red">${data.error}</div>`;
                    return;
                }

                if (data.results.length === 0) {
                    resultsDiv.innerHTML = '<div>No results found</div>';
                    return;
                }

                let html = '';
                data.results.forEach(item => {
                    html += `
                        <div class="result-item">
                            <h3>${item.name}</h3>
                            <p>Date: ${item.date}</p>
                            <p>Type: <span class="result-type ${item.type}">${item.type}</span></p>
                            <p>Path: ${item.file_path}</p>
                        </div>
                    `;
                });

                resultsDiv.innerHTML = html;
            } catch (error) {
                console.error('Error:', error);
                document.getElementById('results').innerHTML = '<div style="color:red">Error fetching results</div>';
            }
        });
    </script>
</body>
</html>
"""

# Flask routes
@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/search')
def search():
    date_from = request.args.get('date_from')
    date_to = request.args.get('date_to')
    imagery_type = request.args.get('imagery_type', 'all')

    if not date_from or not date_to:
        return jsonify({'error': 'Both date ranges are required'}), 400

    try:
        date_from = datetime.strptime(date_from, '%Y-%m-%d').date()
        date_to = datetime.strptime(date_to, '%Y-%m-%d').date()
    except ValueError:
        return jsonify({'error': 'Invalid date format'}), 400

    conn = get_db_connection()
    results = []

    try:
        if imagery_type in ['all', 'professional']:
            professional = conn.execute(
                "SELECT name, date, folder_path as file_path, 'professional' as type "
                "FROM professional WHERE date BETWEEN ? AND ? ORDER BY date",
                (date_from, date_to))
            results.extend(professional.fetchall())

        if imagery_type in ['all', 'personal']:
            personal = conn.execute(
                "SELECT name, date, folder_path as file_path, 'personal' as type "
                "FROM personal WHERE date BETWEEN ? AND ? ORDER BY date",
                (date_from, date_to))
            results.extend(personal.fetchall())

        return jsonify({'results': [dict(row) for row in results]})
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        conn.close()

# Shutdown function
def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()

@app.route('/shutdown', methods=['POST'])
def shutdown():
    shutdown_server()
    return 'Server shutting down...'


def find_free_port():
    """Find a free port starting from 9000"""
    port = 9000
    while True:
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('0.0.0.0', port))
                return port
        except OSError:
            port += 1

def run_flask(port):
    app.run(host='0.0.0.0', port=port, debug=False, use_reloader=False)

# Find an available port
port = find_free_port()

# Start Flask in a thread
flask_thread = threading.Thread(target=run_flask, args=(port,))
flask_thread.daemon = True
flask_thread.start()

# Wait for server to start
time.sleep(2)

# Get proxy URL
proxy_url = output.eval_js(f'google.colab.kernel.proxyPort({port})')

# Display interface with FIXED shutdown button
display(HTML(f"""
    <h2>Drone Imagery Search Interface</h2>
    <a href="{proxy_url}" target="_blank">Open in New Tab</a>
    <iframe src="{proxy_url}" width="100%" height="600px"></iframe>
    <p>To stop the server: <button onclick="fetch('{proxy_url}/shutdown', {{'method': 'POST'}})">Shutdown</button></p>
    <p>Running on port: {port}</p>
"""))

print(f"If the interface doesn't load, visit: {proxy_url}")

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:9001
 * Running on http://172.28.0.12:9001
INFO:werkzeug:Press CTRL+C to quit


If the interface doesn't load, visit: https://9001-m-s-3k7vykytdwftl-b.us-east1-0.prod.colab.dev


## 4. View database content (for debugging purposes)

In [7]:
def show_database_contents():
    """Display database contents for verification"""
    conn = sqlite3.connect("drone_folders.db")

    print("Professional table:")
    df_pro = pd.read_sql_query("SELECT * FROM professional", conn)
    display(df_pro)

    print("\nPersonal table:")
    df_per = pd.read_sql_query("SELECT * FROM personal", conn)
    display(df_per)

    conn.close()

# Uncomment to view database contents
# show_database_contents()

Professional table:


,id,name,date,folder_path
0,1,DJI_202410031255_008_ParadisePondBaseWithLeg,2024-10-03,/content/drive/Shareddrives/Drones - CSC 230/1...
1,2,DJI_202410081539_009_ParadisePondBaseWithLeg,2024-10-08,/content/drive/Shareddrives/Drones - CSC 230/1...
2,3,DJI_202410111314_010_ParadisePondBaseWithLeg,2024-10-11,/content/drive/Shareddrives/Drones - CSC 230/1...
3,4,DJI_202410171306_011,2024-10-17,/content/drive/Shareddrives/Drones - CSC 230/1...
4,5,DJI_202410171306_012_ParadisePondBaseWithLeg,2024-10-17,/content/drive/Shareddrives/Drones - CSC 230/1...
5,6,DJI_202410171502_013_ParadisePondBaseWithLeg,2024-10-17,/content/drive/Shareddrives/Drones - CSC 230/1...
6,7,DJI_202410240928_018_ParadisePondBaseWithLeg,2024-10-24,/content/drive/Shareddrives/Drones - CSC 230/1...



Personal table:


INFO:werkzeug:127.0.0.1 - - [04/May/2025 22:46:52] "GET / HTTP/1.1" 200 -


,id,name,date,folder_path
0,1,10.16CampusWideDrone,2024-10-16,/content/drive/Shareddrives/Drones - CSC 230/1...
1,2,10.22Bio363GooseCounting,2024-10-22,/content/drive/Shareddrives/Drones - CSC 230/1...
2,3,10.23Bahamas2020Exports,2020-10-23,/content/drive/Shareddrives/Drones - CSC 230/1...
3,4,10.16Tacoma,2024-10-16,/content/drive/Shareddrives/Drones - CSC 230/1...
4,5,10.20ConnecticutRiver,2024-10-20,/content/drive/Shareddrives/Drones - CSC 230/1...
5,6,10.28DavisLawn,2024-10-28,/content/drive/Shareddrives/Drones - CSC 230/1...
